In [5]:
import warnings
import numpy as np 
import pandas as pd 
import plotly.express as px
import re
import statsmodels.api as sm
from plotly.subplots import make_subplots


warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/airbnb-listings-reviews/Airbnb Data/Listings.csv
/kaggle/input/airbnb-listings-reviews/Airbnb Data/Reviews.csv
/kaggle/input/airbnb-listings-reviews/Airbnb Data/Reviews_data_dictionary.csv
/kaggle/input/airbnb-listings-reviews/Airbnb Data/Listings_data_dictionary.csv


In [2]:
df = pd.read_csv('/kaggle/input/airbnb-listings-reviews/Airbnb Data/Listings.csv', encoding='unicode_escape')
df.head(2)

,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
0,281420,"Beautiful Flat in le Village Montmartre, Paris",1466919,2011-12-03,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
1,3705183,39 mÃÂ² Paris (Sacre CÃ âur),10328771,2013-11-29,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f


<link href="https://fonts.googleapis.com/css2?family=Vazirmatn&display=swap" rel="stylesheet">
<div style="background-color: #ffddd2; color: black; width: 100%; float: left; box-sizing: border-box; padding: 15px; font-family: 'Vazirmatn', sans-serif;">
    <p style="font-size: 20px; text-align: right; font-weight: bold;">دشبورد ساده ای شامل چند متریک طراحی کنید که موارد زیر را نمایش دهد</h2>
    <p style="font-size: 16px; text-align: right; font-weight: 500;">الف) توزیع میزبانها در شهرهای مختلف</p>
    <p style="font-size: 16px; text-align: right; font-weight: 500;">ب) تنوع میزبانها از نظر تعداد اقامتگاه و طول عمر فعالیتشان</p>
    <p style="font-size: 16px; text-align: right; font-weight: 500;">پ) میزان پاسخگویی میزبان ها</p> 
</div>


In [10]:
host_distribution= df.groupby('city')[['host_id']].nunique().reset_index()
host_distribution.rename(columns={'host_id':'count'}, inplace=True)
host_distribution.sort_values('count', inplace=True)
fig1 = px.bar(host_distribution, x="city", y="count",color = 'count', 
            color_continuous_scale = 'Burg', title='توزیع میزبان‌ها در شهرهای مختلف')
fig1.update_layout(
    title_font_family="Vazir",
    title_x = 0.5,
    showlegend=False
)

grouped_by_host = df.groupby('host_id').count().reset_index()[['host_id', 'listing_id']]
grouped_by_host.rename(columns={'listing_id':'Number of residences'}, inplace=True)
value1 = grouped_by_host[grouped_by_host['Number of residences'] == 1].count()['Number of residences']
value2 = grouped_by_host[grouped_by_host['Number of residences'] == 2].count()['Number of residences']
value3 = grouped_by_host[grouped_by_host['Number of residences'] > 2][grouped_by_host['Number of residences'] < 11].count()['Number of residences']
value4 = grouped_by_host[grouped_by_host['Number of residences'] >= 11].count()['Number of residences']

data = [[value1, '1'], [value2, '2'], [value3, 'between 2 and 10'], [value4, 'more than 10']]
grouped_by_host_df = pd.DataFrame(data,columns = ['count of host', 'number of residences'])

fig2 = px.pie(grouped_by_host_df, values='count of host', names='number of residences', title = 'نسبت میزبان‌ها براساس تعداد اقامتگاه‌هایشان', color_discrete_sequence = px.colors.sequential.RdBu)
fig2.update_layout(title_font_family="Vazir", title_x = 0.5)

host_response_time = df[['host_response_time', 'host_id']].drop_duplicates(subset = 'host_id').groupby('host_response_time').count().reset_index()
host_response_time.rename(columns={'host_id':'count'}, inplace=True)
fig3 = px.pie(host_response_time, values='count', names='host_response_time', color_discrete_sequence = px.colors.sequential.RdBu, title='میزان پاسخگویی میزبان‌ها')
fig3.update_layout(
    title_font_family="Vazir",
    title_x = 0.5
)

fig1.show()

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "domain"}, {"type": "domain"}]], subplot_titles=("نسبت میزبان‌ها براساس تعداد اقامتگاه‌هایشان", "میزان پاسخگویی میزبان‌ها"))

for trace in fig2.data:
    fig.add_trace(trace, row=1, col=1)
for trace in fig3.data:
    fig.add_trace(trace, row=1, col=2)

fig.update_traces(marker=dict(colors=px.colors.sequential.RdBu))

fig.update_layout(title_x=0.5, height=500)
fig.layout.annotations[0].update(font=dict(family="Vazir", size=16), y=1.1)
fig.layout.annotations[1].update(font=dict(family="Vazir", size=16), y=1.1)
fig.show()

<link href="https://fonts.googleapis.com/css2?family=Vazirmatn&display=swap" rel="stylesheet">
<div style="background-color: #ffddd2; color: black; width: 100%; float: left; box-sizing: border-box; padding: 15px; font-family: 'Vazirmatn', sans-serif;">
    <p style="font-size: 16px; text-align: right; font-weight: 500;">
فرض کنید میخواهیم میزان عرضه اقامتگاههای مختلف بر اساس را در شهرهای مختلف مقایسه کنیم دو متریک مختلف برای این منظور تعریف کنید که یکی مطلق و دیگری نسبی باشد. حال با استفاده از این دو متریک سعی کنید میزان عرضه ی اتاق اختصاصی و اتاق اشتراکی را در پاریس و نیویورک با هم مقایسه کنید 
    </p>
</div>




In [6]:
q2 = df[df['room_type'].isin(['Private room', 'Shared room'])].groupby(['city', 'room_type']).count().reset_index()
q2.rename(columns={'listing_id':'count'}, inplace=True)

fig4 = px.pie(q2[q2['city'] == 'Paris'], values='count', names='room_type', title='مقایسه اقامتگاه‌ها در پاریس', color_discrete_sequence=px.colors.sequential.RdBu)
fig5 = px.pie(q2[q2['city'] == 'New York'], values='count', names='room_type', title='مقایسه اقامتگاه‌ها در نیویورک', color_discrete_sequence=px.colors.sequential.RdBu)

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "domain"}, {"type": "domain"}]], subplot_titles=("مقایسه اقامتگاه‌ها در پاریس", "مقایسه اقامتگاه‌ها در نیویورک"))

for trace in fig4.data:
    fig.add_trace(trace, row=1, col=1)
for trace in fig5.data:
    fig.add_trace(trace, row=1, col=2)

fig.update_traces(marker=dict(colors=px.colors.sequential.RdBu))

fig.update_layout(title_x=0.5, height=500)
fig.layout.annotations[0].update(font=dict(family="Vazir", size=16), y=1.1)
fig.layout.annotations[1].update(font=dict(family="Vazir", size=16), y=1.1)

pr_ny = q2[q2['city'].isin(['Paris', 'New York'])]
fig6 = px.bar(pr_ny, x='room_type', y = 'count', color='city', barmode='group', height = 500, title='تعداد اقامتگاه‌ها براساس نوع اقامتگاه برای شهرهای پاریس و نیویورک',  color_discrete_sequence = px.colors.sequential.RdBu)
fig6.update_layout(
    title_font_family="Vazir",
    title_x = 0.5
)
print()

<link href="https://fonts.googleapis.com/css2?family=Vazirmatn&display=swap" rel="stylesheet">
<div style="background-color: #FFFFFF; color: black; width: 100%; float: left; box-sizing: border-box; padding: 15px; font-family: 'Vazirmatn', sans-serif; direction: rtl;">
    <p style="font-size: 16px; text-align: right; font-weight: 500;">
        متریک <span style="font-family: Arial, sans-serif;">absolute</span>: تعداد اقامتگاه‌ها یک <span style="font-family: Arial, sans-serif;">room type</span> در آن شهر<br>
        متریک <span style="font-family: Arial, sans-serif;">ratio</span>: تعداد اقامتگاه‌ها یک <span style="font-family: Arial, sans-serif;">room type</span> شهر نسبت به تعداد کل اقامتگاه‌های آن شهر<br>
        اگر بخواهیم تعداد اقامتگاه‌‌های شهرهای پاریس و نیویورک را بررسی کنیم، در <span style="font-family: Arial, sans-serif;">room type</span>های انتخاب شده شهر نیویورک تعداد اقامتگاه‌های بیشتری دارد. اما با بررسی نسبت هر <span style="font-family: Arial, sans-serif;">room type</span> به کل اقامتگاه‌های شهر، میتوان به این نتیجه رسید که نسبت <span style="font-family: Arial, sans-serif;">room type</span>های مختلف در دو شهر تقریبا برابر است.
    </p>
</div>

In [7]:
fig.show()
fig6.show()